Recreating the Adaptive SEIR Model presented in S Dobson's book 'Epidemic modelling – Some notes, maths, and code' under the 'Asymptomatic transmission' chapter.

Link to book: https://simondobson.org/introduction-to-epidemics/seir.html

Link to repo: https://github.com/simoninireland/introduction-to-epidemics/blob/master/src/seir.ipynb

In [6]:
from epyc import JSONLabNotebook, ParallelLab
import epyc
import epydemic
import numpy as np
from epydemic import SEIR, SIR, ERNetwork, Monitor, ProcessSequence, rng, SynchronousDynamics
from parameters import *

In [7]:
create_data_output_directory()

The following class extends the standard SEIR model to use an additional ```def quarantine(self, n)``` function. This function will rewire a fraction of SI edges to an infected node thus removing a proportion (```P_REWIRE```) of connections with adjacent susceptible neighbors.  

In [8]:
# The code below was taken from a GitHub repo by Simon Dobson at https://github.com/simoninireland/introduction-to-epidemics/blob/master/src/seir.ipynb (last accessed 2023-11-09) 
class AdaptiveSEIR(SEIR):
    P_DETECT = 'pDetect'  #: Parameter for the probability of detecting an exposed neighbour of an infected node.
    P_REWIRE = 'pRewire'  #: Parameter for the probability of rewiring an SE or SI edge.

    def __init__(self):
        super(AdaptiveSEIR, self).__init__()
        self._pDetect = None
        self._pRewire = None

    def build(self, params):
        super(AdaptiveSEIR, self).build(params)

        # store the parameters for later
        self._pDetect = params[self.P_DETECT]
        self._pRewire = params[self.P_REWIRE]

        # also monitor other compartments
        self.trackNodesInCompartment(epydemic.SEIR.SUSCEPTIBLE)
        self.trackNodesInCompartment(epydemic.SEIR.REMOVED)

    def quarantine(self, n):
        g = self.network()
        rng = np.random.default_rng()
        ms = list(g.neighbors(n))
        for m in ms:
            if self.getCompartment(m) == self.SUSCEPTIBLE and rng.random() <= self._pRewire:
                # a susceptible neighbour, remove link to us
                self.removeEdge(n, m)

                # rewire to another random susceptible
                mprime = self.locus(self.SUSCEPTIBLE).draw()
                self.addEdge(m, mprime)

    def symptoms(self, t, n):
        # perform a normal becoming-symptomatic event
        super(AdaptiveSEIR, self).symptoms(t, n)

        g = self.network()
        rng = np.random.default_rng()

        # examine all neighbours and look for exposed
        # nodes to quarantine
        ms = list(g.neighbors(n))
        for m in ms:
            if self.getCompartment(m) == self.EXPOSED and rng.random() <= self._pDetect:
                # detected an exposed individual,
                # quarantine them
                self.quarantine(m)

        # quarantine the newly symptomatic node
        self.quarantine(n)
# END Copied Code

In [9]:
def ex_1_original_seir(lab):
    # set the disease parameter space
    lab[SEIR.P_EXPOSED] = p_exposed
    lab[SEIR.P_INFECT_SYMPTOMATIC] = p_infect
    lab[SEIR.P_INFECT_ASYMPTOMATIC] = p_infect
    lab[SEIR.P_REMOVE] = p_remove
    lab[SEIR.P_SYMPTOMS] = p_remove
        
    lab[AdaptiveSEIR.P_REWIRE] = p_rewire
    lab[AdaptiveSEIR.P_DETECT] = p_detect

    lab[epydemic.Monitor.DELTA] = delta
    
    lab['ens'] = range(ens)

    # set the topology for the generated network
    lab[ERNetwork.N] = n
    lab[ERNetwork.KMEAN] = k_mean

    # create the model, network generator, and experiment
    p = AdaptiveSEIR()
    g = ERNetwork()
    e = epydemic.StochasticDynamics(p, g)

    # run the experiment
    lab.runExperiment(e)

In [10]:
lab = ParallelLab(JSONLabNotebook(get_out_path("basic-seir-data"), create=True), cores)
lab.createWith(get_out_path("basic-seir-data"), ex_1_original_seir)

True